<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_9_2__rekomendowanie_hoteli_langchain_creaw_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qqU langchain_community crewai crewai_tools gradio

Kod instaluje kilka pakietów Pythona za pomocą narzędzia pip:

- langchain_community - biblioteka do pracy z modelami językowymi i łączenia ich w złożone łańcuchy przetwarzania
- crewai - framework do tworzenia zespołów agentów AI współpracujących ze sobą
- crewai_tools - zestaw narzędzi rozszerzających możliwości crewai
- gradio - biblioteka do tworzenia interfejsów graficznych dla modeli AI

Flagi użyte w komendzie:
- -qq - bardzo cichy tryb instalacji (quiet quiet), ograniczający wyświetlane komunikaty do minimum
- -U - aktualizacja pakietów do najnowszych wersji, jeśli są już zainstalowane

Jest to pojedyncza komenda instalacyjna łącząca instalację wszystkich potrzebnych pakietów w jednym kroku.

In [ ]:
# Standard library imports
import os
import warnings

# Third-party imports
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain_community.llms import Ollama
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

from IPython.display import Markdown

from google.colab import userdata

import json
# Suppress warnings
warnings.filterwarnings('ignore')

import gradio as gr

from pprint import pp as ppr


Ten kod importuje różne biblioteki i moduły potrzebne do działania aplikacji:

Ze standardowej biblioteki Pythona:
- os - do operacji na systemie operacyjnym
- warnings - do zarządzania ostrzeżeniami (które są tutaj wyłączone)

Z zewnętrznych bibliotek:
- crewai - importuje główne komponenty: Agent, Task i Crew oraz Process
- crewai_tools - narzędzia do wyszukiwania (SerperDev) i scrapowania stron
- dotenv - do wczytywania zmiennych środowiskowych
- langchain - komponenty do pracy z modelami językowymi (Ollama, ChatOpenAI)
- Document z langchain_core - do pracy z dokumentami tekstowymi
- Markdown z IPython.display - do formatowania tekstu w notebookach
- userdata z google.colab - do obsługi danych użytkownika w Google Colab
- json - do pracy z danymi w formacie JSON
- gradio - do tworzenia interfejsów graficznych
- pprint jako ppr - do ładnego wyświetlania złożonych struktur danych

Kod konfiguruje także środowisko poprzez wyłączenie ostrzeżeń za pomocą warnings.filterwarnings('ignore').

In [ ]:
class CFG:
    model = 'gpt-4o-mini'
    temp = 0

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')
os.environ["SERPER_API_KEY"] = userdata.get('serper')

# Załoga

In [ ]:
def run_travel_agent_crew(location, check_in, check_out, number_of_people, budget):

    # llm
    llm = ChatOpenAI(model= CFG.model, temperature = CFG.temp)

    # tools
    serper_search_tool = SerperDevTool()
    scrap_tool = ScrapeWebsiteTool()
    website_search_tool = WebsiteSearchTool()
    tools=[serper_search_tool, scrap_tool, website_search_tool]

    # agent definitions
    travel_expert = Agent( role='Specjalista ds. Hoteli',
      goal='Sprawnie wyszukiwać i tworzyć kompleksową listę odpowiednich opcji hotelowych w określonej lokalizacji  \
      przestrzegając podanych kryteriów, takich jak daty zameldowania/wymeldowania, liczba gości i wszelkie inne szczególne wymagania. \
      Zapewnić, że zebrane informacje są dokładne, aktualne i zawierają kluczowe szczegóły, jak ceny, udogodnienia i oceny gości.',
      backstory='Jesteś doświadczonym cyfrowym concierge podróży z wyczuleniem na szczegóły i pasją do znajdowania idealnego zakwaterowania. \
      Dzięki wieloletniemu doświadczeniu w branży hotelarskiej i rozległej wiedzy o światowych sieciach hoteli oraz butikowych obiektach, \
      doskonalisz swoje umiejętności w poruszaniu się po różnych platformach rezerwacyjnych i bazach danych hoteli. \
      Twoja specjalizacja polega na szybkim przeglądaniu licznych opcji w celu identyfikacji najbardziej odpowiednich wyborów dla podróżnych, \
      biorąc pod uwagę czynniki takie jak lokalizacja, cena, udogodnienia i opinie gości.',
      verbose=True, allow_delegation=False, llm=llm, tools=tools)
    hotel_reviewer = Agent(
      role='Analityk Jakości Hoteli',
      goal='Dokładnie oceniać i porównywać opcje hotelowe, aby dostarczać szczegółowych, bezstronnych ocen pomagających podróżnym w podejmowaniu świadomych decyzji. \
      Analizować udogodnienia, lokalizację, stosunek jakości do ceny i doświadczenia gości każdego hotelu, aby tworzyć kompleksowe recenzje podkreślające mocne strony, \
      słabości i unikalne cechy.',
      backstory = "Jesteś doświadczonym profesjonalistą branży turystycznej z ponad dziesięcioletnim doświadczeniem w ocenie i krytyce hoteli. \
      Twoje doświadczenie obejmuje pracę jako inspektor hoteli luksusowych, dziennikarz turystyczny dla znanych publikacji oraz konsultant systemów oceny hoteli.  \
      To różnorodne doświadczenie rozwinęło Twoją zdolność do oceny zakwaterowania z wielu perspektyw, uwzględniając zarówno obiektywne kryteria, \
      jak i subiektywne doświadczenia gości.   Masz szczególny talent do identyfikowania ukrytych perełek i dostrzegania potencjalnych problemów, które mogą wpłynąć na pobyt gościa.",
      verbose=True, allow_delegation=False, llm=llm, tools=tools)

    # tasks
    hotel_search_task = Task(
            description = "Wyszukaj 5 hoteli w {location} dla {number_of_people} osób dorosłych, z zameldowaniem {check_in} i wymeldowaniem {check_out}.",
            agent = travel_expert, expected_output = "Wszystkie szczegóły dotyczące konkretnie wybranego zakwaterowania."
      )

    hotel_review_task = Task(
        description="Na podstawie dostarczonych rekomendacji, wybierz najlepsze opcje w oparciu o oceny, recenzje i dostępne udogodnienia. \
        Weź pod uwagę, że budżet wynosi {budget} PLN. Postaraj się znaleźć zakwaterowanie w granicach tego budżetu.",
        expected_output="Wszystkie szczegóły dotyczące konkretnie wybranego zakwaterowania, w tym cena, URL i ewentualne zdjęcia, jeśli są dostępne.",
        agent = hotel_reviewer)


    # Create the Crew
    travel_agent_crew = Crew( agents=[travel_expert, hotel_reviewer],
    tasks=[hotel_search_task, hotel_review_task],
    verbose=True, process=Process.sequential)
    event_criteria = {
        'location': location,
        'check_in': check_in,
        'check_out': check_out,
        'number_of_people': int(number_of_people),
        'budget': float(budget)
    }

    result = travel_agent_crew.kickoff(inputs=event_criteria)

    # Convert the result to Markdown
    markdown_result = Markdown(result.raw).data
    return markdown_result


Ten kod definiuje funkcję tworzącą zespół agentów AI do wyszukiwania i oceny hoteli. Oto jak działa:

Konfiguracja narzędzi:
- Tworzy model językowy ChatOpenAI z określonymi parametrami
- Inicjalizuje narzędzia do wyszukiwania (SerperDev), scrapowania stron i przeszukiwania witryn

Definiuje dwóch agentów:
1. Specjalista ds. Hoteli - ekspert w wyszukiwaniu opcji hotelowych, znający się na platformach rezerwacyjnych i bazach danych hoteli

2. Analityk Jakości Hoteli - ocenia i porównuje hotele, analizując ich udogodnienia, lokalizację i stosunek jakości do ceny

Tworzy dwa zadania:
1. Wyszukanie 5 hoteli spełniających kryteria (lokalizacja, liczba osób, daty)
2. Analiza znalezionych opcji pod kątem ocen, recenzji i udogodnień w ramach określonego budżetu

Zespół (Crew):
- Łączy agentów i zadania w sekwencyjny proces
- Przyjmuje parametry wyszukiwania (miejsce, daty, liczba osób, budżet)
- Uruchamia proces wyszukiwania i analizy
- Zwraca wyniki w formacie Markdown

Funkcja run_travel_agent_crew jest głównym interfejsem, który koordynuje całą operację i przekształca surowe wyniki w sformatowany tekst.

In [ ]:
# Create Gradio interface
iface = gr.Interface(
    fn=run_travel_agent_crew,
    inputs=[
        gr.Textbox(label="Location"),
        gr.Textbox(label="Check-in Date"),
        gr.Textbox(label="Check-out Date"),
        gr.Number(label="Number of People"),
        gr.Number(label="Budget")
    ],
    outputs=gr.Markdown(label="Result"),
    title="Travel Agent Crew",
    description="Enter your travel details to get hotel recommendations.",
    allow_flagging="never",
)

# Launch the app
iface.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b3ae91cd39dc4b8570.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Agent: Specjalista ds. Hoteli
## Task: Wyszukaj 5 hoteli w Lublin dla 4 osób dorosłych, z zameldowaniem 2024-12-22 i wymeldowaniem 2024-12-27.


# Agent: Specjalista ds. Hoteli
## Thought: Muszę wyszukać hotele w Lublinie, które mogą pomieścić 4 dorosłych, z datami zameldowania 22 grudnia 2024 i wymeldowania 27 grudnia 2024. Rozpocznę od przeszukania internetu w celu znalezienia odpowiednich opcji.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"hotele w Lublinie dla 4 os\\u00f3b doros\\u0142ych, zameldowanie 2024-12-22, wymeldowanie 2024-12-27\"}"
## Tool Output: 

Search results: Title: Noclegi Komfort, Lublin (aktualne ceny na rok 2024)
Link: https://www.booking.com/hotel/pl/pokoje-goscinne-komfort.pl.html
Snippet: Obiekt Noclegi Komfort znajduje się w miejscowości Lublin i oferuje bezpłatne WiFi, bezpłatny prywatny parking oraz widok na miasto.
---
Title: Apartament Active Mercure, Lublin (aktualne ceny na rok 2024)
Link: https://www.booking.com/hotel/pl/

Ten kod tworzy interfejs graficzny aplikacji za pomocą biblioteki Gradio:

Definiuje formularz z polami:
- Location - pole tekstowe na miejsce docelowe
- Check-in Date - data zameldowania
- Check-out Date - data wymeldowania  
- Number of People - pole numeryczne na liczbę osób
- Budget - pole numeryczne na budżet

Konfiguracja interfejsu:
- Używa funkcji run_travel_agent_crew jako głównej funkcji przetwarzającej
- Wyświetla wyniki w formacie Markdown
- Nadaje tytuł "Travel Agent Crew"
- Dodaje opis "Enter your travel details to get hotel recommendations"
- Wyłącza możliwość flagowania wyników (allow_flagging="never")

iface.launch(debug=True) uruchamia aplikację w trybie debugowania, co pozwala na śledzenie ewentualnych błędów podczas jej działania.

# Kroki w nieznane


*   Deployment: Spaces https://huggingface.co/spaces
*   Tutorial: https://www.gradio.app/guides/quickstart

